# AAI-540 Group 6 Project<br>
# VenueSignal<br>
<br>

#1) Setup, Configuration, and Connect 

In [1]:
!python --version

Python 3.12.9


In [2]:
!pip list

Package                                   Version
----------------------------------------- -----------
absl-py                                   2.3.1
accelerate                                1.12.0
adagio                                    0.2.6
aioboto3                                  14.3.0
aiobotocore                               2.22.0
aiofiles                                  25.1.0
aiohappyeyeballs                          2.6.1
aiohttp                                   3.13.2
aiohttp-cors                              0.8.1
aioitertools                              0.12.0
aiosignal                                 1.4.0
aiosqlite                                 0.19.0
alembic                                   1.17.2
altair                                    5.5.0
amazon-q-developer-jupyterlab-ext         3.4.8
amazon_sagemaker_jupyter_ai_q_developer   1.2.8
amazon_sagemaker_jupyter_scheduler        3.1.15
amazon-sagemaker-sql-editor               0.1.19
amazon-sagemaker-sql-e

In [3]:
!pip install --disable-pip-version-check -q pip --upgrade > /dev/null
!pip install --disable-pip-version-check -q wrapt==1.17.2
!pip install --disable-pip-version-check -q sparkmagic==0.22.0
!pip install --disable-pip-version-check -q nvidia-ml-py3==7.352.0
!pip install --disable-pip-version-check -q pydynamodb

In [4]:
!pip install --disable-pip-version-check -q awscli==1.18.216 boto3==1.29.6 botocore==1.37.2

ERROR: Cannot install awscli==1.18.216 and botocore==1.37.2 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [5]:
!pip install --disable-pip-version-check -q botocore==1.37.4

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.22.0 requires botocore<1.37.4,>=1.37.2, but you have botocore 1.37.4 which is incompatible.
boto3 1.42.34 requires botocore<1.43.0,>=1.42.34, but you have botocore 1.37.4 which is incompatible.
awscli 1.44.24 requires botocore==1.42.34, but you have botocore 1.37.4 which is incompatible.


In [6]:
!pip install --disable-pip-version-check -q awswrangler

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.22.0 requires botocore<1.37.4,>=1.37.2, but you have botocore 1.42.34 which is incompatible.


In [7]:
!pip list

Package                                   Version
----------------------------------------- -----------
absl-py                                   2.3.1
accelerate                                1.12.0
adagio                                    0.2.6
aioboto3                                  14.3.0
aiobotocore                               2.22.0
aiofiles                                  25.1.0
aiohappyeyeballs                          2.6.1
aiohttp                                   3.13.2
aiohttp-cors                              0.8.1
aioitertools                              0.12.0
aiosignal                                 1.4.0
aiosqlite                                 0.19.0
alembic                                   1.17.2
altair                                    5.5.0
amazon-q-developer-jupyterlab-ext         3.4.8
amazon_sagemaker_jupyter_ai_q_developer   1.2.8
amazon_sagemaker_jupyter_scheduler        3.1.15
amazon-sagemaker-sql-editor               0.1.19
amazon-sagemaker-sql-e

In [8]:
setup_dependencies_passed = True

In [9]:
%store setup_dependencies_passed

Stored 'setup_dependencies_passed' (bool)


In [10]:
%store

Stored variables and their in-db values:
setup_dependencies_passed             -> True


In [11]:
pip install --upgrade boto3 botocore awscli

Note: you may need to restart the kernel to use updated packages.


In [12]:
#@title Setup and Import libraries
import pandas as pd
import numpy as np
import os
import json
import re
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# AWS
import boto3
import sagemaker
from pyathena import connect
from pyathena.pandas.cursor import PandasCursor

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("Libraries imported successfully")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
Libraries imported successfully


In [13]:
#@title S3 Bucket Configuration
# The bucket is located at s3://yelp-aai540-group6/yelp-dataset/
# The ARN for the bucket is arn:aws:s3:::yelp-aai540-group6
S3_BUCKET = 'yelp-aai540-group6'
DATABASE_NAME = 'yelp-dataset'


In [14]:
#@title Athena Configuration
ATHENA_OUTPUT = f's3://{S3_BUCKET}/athena-results/'
REGION = 'us-east-1'  

In [15]:
from IPython.display import display, HTML

display(
    HTML(
        f'<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{S3_BUCKET}/yelp-dataset/json/?region={REGION}&tab=overview">S3 Bucket - Yelp Dataset</a></b>'
    )
)

# Release Resources

In [ ]:

%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}